In [1]:
!pip install torch
!pip install transformers
!pip install pandas
!pip install scikit-learn

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm.notebook import tqdm

In [6]:
data = pd.read_csv('../raw data/combined_data.csv')
data.head()

,Text,Label,Model,Original dataset,Row in original dataset
0,While driverless cars present many promising b...,Machine,darragh_claude_v7,essays,13355
1,Homework Clubs: The Key to Unlocking Academic ...,Machine,llama2_chat,essays,7249
2,"""The legalization of marijuana has been a cont...",Machine,falcon_180b_v1,essays,2603
3,Taking the opportunity to learn new things can...,Machine,mistral7binstruct_v1,essays,3993
4,Working with a partner is an effective way fo...,Machine,mistral7binstruct_v2,essays,3773


In [7]:
data['Label'] = data['Label'].apply(lambda x: 0 if x == 'Human' else 1)

In [8]:
sampled_data = data.sample(n=200, random_state=42)

In [9]:
train_X, test_X, train_Y, test_Y = train_test_split(
    sampled_data['Text'], 
    sampled_data['Label'], 
    train_size=0.7, 
    random_state=42
)

# Print the shapes to verify
print(f"Train X shape: {train_X.shape}")
print(f"Test X shape: {test_X.shape}")
print(f"Train Y shape: {train_Y.shape}")
print(f"Test Y shape: {test_Y.shape}")

Train X shape: (140,)
Test X shape: (60,)
Train Y shape: (140,)
Test Y shape: (60,)


In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [11]:
train_tokens = tokenizer(list(train_X), padding = True, truncation=True)
test_tokens = tokenizer(list(test_X), padding = True, truncation=True)

In [12]:
train_tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [13]:
print(train_tokens['input_ids'][0])
print(tokenizer.decode(train_tokens['input_ids'][0]))

[101, 6510, 1431, 1129, 1682, 1106, 1902, 1147, 1319, 2247, 1933, 1272, 117, 1122, 1660, 1172, 170, 2640, 1106, 1138, 1380, 1106, 1440, 1977, 1106, 1107, 1103, 2247, 119, 2907, 1122, 1156, 1494, 1172, 1243, 170, 2640, 1120, 13992, 1147, 1319, 17980, 119, 6510, 1156, 1129, 1682, 1106, 1138, 1115, 2296, 1115, 1152, 1132, 1682, 1106, 1321, 1113, 1103, 4812, 119, 1220, 1156, 1129, 1682, 1106, 3858, 1167, 1191, 1152, 1169, 2011, 1122, 1155, 1113, 1147, 1319, 119, 1370, 1538, 1651, 1152, 1274, 1204, 1541, 1440, 1977, 1106, 1833, 1933, 1133, 1191, 1128, 1660, 1172, 170, 2640, 1106, 1202, 1122, 1113, 1147, 1319, 1152, 1547, 1243, 1154, 1122, 119, 1752, 117, 13795, 1240, 1319, 1933, 1112, 170, 2377, 1122, 1660, 1128, 1380, 1106, 1440, 1977, 1106, 1107, 1103, 2247, 119, 2082, 1651, 2215, 1313, 1111, 1103, 2247, 1114, 1720, 1106, 1202, 119, 1573, 117, 1191, 1128, 1660, 170, 2377, 170, 2640, 1106, 1902, 1147, 1319, 1933, 1111, 1103, 2247, 1122, 1660, 1172, 170, 2640, 1106, 1202, 1380, 119, 6510, 1

In [14]:
class TokenData(Dataset):
    def __init__(self, train = False):
        if train:
            self.text_data = train_X
            self.tokens = train_tokens
            self.labels = list(train_Y)
        else:
            self.text_data = test_X
            self.tokens = test_tokens
            self.labels = list(test_Y)

    def __len__(self):
        return len(self.text_data)

    def __getitem__(self, idx):
        sample = {}
        for k, v in self.tokens.items():
            sample[k] = torch.tensor(v[idx])
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

In [15]:
batch_size = 2
train_dataset = TokenData(train = True)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = TokenData(train = False)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

In [16]:
bert_model = BertForSequenceClassification.from_pretrained('bert-base-cased') # Pre-trained model
optimizer = AdamW(bert_model.parameters(), lr=1e-5) # Optimization function
loss_fn = torch.nn.CrossEntropyLoss() # Loss function

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
num_epochs = 3
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
bert_model.to(device) # Transfer model to GPU if available
print(f"Using {device}")

Using cpu


In [23]:
for epoch in range(num_epochs):
    print("Epoch: ",(epoch + 1))
    # TRAINING BLOCK STARTS
    bert_model.train()
    for i,batch in tqdm(enumerate(train_loader)):    
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Setting the gradients to zero
        optimizer.zero_grad()
        
        # Passing the data to the model
        outputs = bert_model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'])
        
        # The logits will be used for measuring the loss
        pred = outputs.logits
        loss = loss_fn(pred, batch['labels'])

        # Calculating the gradient for the loss function
        loss.backward()
        
        # Optimizing the parameters of the bert model
        optimizer.step()

        # Calculating the running loss for logging purposes
        train_batch_loss = loss.item()
        train_last_loss = train_batch_loss / batch_size

        #print('Training batch {} last loss: {}'.format(i + 1, train_last_loss))
    # Logging epoch-wise training loss
    print(f"\nTraining epoch {epoch + 1} loss: ",train_last_loss)
    # TRAINING BLOCK ENDS 

    # TESTING BLOCK STARTS
    bert_model.eval()
    correct = 0
    test_pred = []
    for i, batch in enumerate(test_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # We don't need gradients for testing
        with torch.no_grad():
            outputs = bert_model(input_ids = batch['input_ids'], attention_mask = batch['attention_mask'])
        
        # Logits act as predictions
        logits = outputs.logits
        
        # Calculating total batch loss using the logits and labels
        loss = loss_fn(logits, batch['labels'])
        test_batch_loss = loss.item()
        
        # Calculating the mean batch loss
        test_last_loss = test_batch_loss / batch_size
        print('Testing batch {} loss: {}'.format(i + 1, test_last_loss))
        
        # Comparing the predicted target with the labels in the batch
        correct += (logits.argmax(1) == batch['labels']).sum().item()
        print("Testing accuracy: ",correct/((i + 1) * batch_size))
    
    print(f"\nTesting epoch {epoch + 1} last loss: ",test_last_loss)
    # TESTING BLOCK ENDS

Epoch:  1


0it [00:00, ?it/s]


Training epoch 1 loss:  0.2695750892162323
Testing batch 1 loss: 0.35671645402908325
Testing accuracy:  0.5
Testing batch 2 loss: 0.27578186988830566
Testing accuracy:  0.75
Testing batch 3 loss: 0.3001209795475006
Testing accuracy:  0.6666666666666666
Testing batch 4 loss: 0.2998211085796356
Testing accuracy:  0.75
Testing batch 5 loss: 0.24831734597682953
Testing accuracy:  0.8
Testing batch 6 loss: 0.3468441963195801
Testing accuracy:  0.75
Testing batch 7 loss: 0.4780895709991455
Testing accuracy:  0.6428571428571429
Testing batch 8 loss: 0.2938806414604187
Testing accuracy:  0.6875
Testing batch 9 loss: 0.28955894708633423
Testing accuracy:  0.6666666666666666
Testing batch 10 loss: 0.39042699337005615
Testing accuracy:  0.6
Testing batch 11 loss: 0.32081276178359985
Testing accuracy:  0.5909090909090909
Testing batch 12 loss: 0.2816104590892792
Testing accuracy:  0.625
Testing batch 13 loss: 0.4543875455856323
Testing accuracy:  0.5769230769230769
Testing batch 14 loss: 0.236099

0it [00:00, ?it/s]


Training epoch 2 loss:  0.12803222239017487
Testing batch 1 loss: 0.27118226885795593
Testing accuracy:  0.5
Testing batch 2 loss: 0.20550091564655304
Testing accuracy:  0.75
Testing batch 3 loss: 0.2530181407928467
Testing accuracy:  0.6666666666666666
Testing batch 4 loss: 0.4214833974838257
Testing accuracy:  0.625
Testing batch 5 loss: 0.6599513292312622
Testing accuracy:  0.5
Testing batch 6 loss: 0.499452143907547
Testing accuracy:  0.5
Testing batch 7 loss: 0.2641464173793793
Testing accuracy:  0.5
Testing batch 8 loss: 0.16828963160514832
Testing accuracy:  0.5625
Testing batch 9 loss: 0.14022141695022583
Testing accuracy:  0.6111111111111112
Testing batch 10 loss: 0.29820340871810913
Testing accuracy:  0.65
Testing batch 11 loss: 0.35010895133018494
Testing accuracy:  0.6363636363636364
Testing batch 12 loss: 0.40451470017433167
Testing accuracy:  0.625
Testing batch 13 loss: 0.2504422962665558
Testing accuracy:  0.6538461538461539
Testing batch 14 loss: 0.1891726702451706
Te

0it [00:00, ?it/s]


Training epoch 3 loss:  0.040363069623708725
Testing batch 1 loss: 0.7748287916183472
Testing accuracy:  0.5
Testing batch 2 loss: 0.032601311802864075
Testing accuracy:  0.75
Testing batch 3 loss: 0.45634421706199646
Testing accuracy:  0.6666666666666666
Testing batch 4 loss: 0.41449975967407227
Testing accuracy:  0.625
Testing batch 5 loss: 0.23319190740585327
Testing accuracy:  0.6
Testing batch 6 loss: 0.1057242676615715
Testing accuracy:  0.6666666666666666
Testing batch 7 loss: 0.1876138597726822
Testing accuracy:  0.7142857142857143
Testing batch 8 loss: 0.13964402675628662
Testing accuracy:  0.75
Testing batch 9 loss: 0.07507424056529999
Testing accuracy:  0.7777777777777778
Testing batch 10 loss: 0.053340066224336624
Testing accuracy:  0.8
Testing batch 11 loss: 0.7253049612045288
Testing accuracy:  0.7727272727272727
Testing batch 12 loss: 0.4323389530181885
Testing accuracy:  0.75
Testing batch 13 loss: 0.6508691310882568
Testing accuracy:  0.7307692307692307
Testing batch 